In [38]:
def parseReviewCorpus():
    from konlpy.tag import Kkma
    twitter = Twitter()
    kkma = Kkma()

    data = open('./review_corpus/ratings_test.txt','r',encoding='UTF-8')

    output = open("review_data.txt", 'w',encoding='UTF-8')

    header = data.readline().split('\n')[0].split('\t')

    for review in data:
        rv = review.split('\n')[0].split('\t')
        rv[1] = kkma.pos(rv[1])
        t=[]
        for item in rv[1]:
            t.append("/".join(item))
        rv[1] = " ".join(t)
        output.write("\t".join(rv)+'\n')

#     for review in test_data:
#         rv = review.split('\n')[0].split('\t')
#         rv[1] = kkma.pos(rv[1])
#         t=[]
#         for item in rv[1]:
#             t.append("/".join(item))
#         rv[1] = " ".join(t)
#         test.write("\t".join(rv)+'\n')

    data.close()
    output.close()

In [39]:
parseReviewCorpus()

java.lang.OutOfMemoryErrorPyRaisable: java.lang.OutOfMemoryError: Java heap space

In [46]:
review_data = open("review_data.txt", 'r',encoding='UTF-8')

data = []
target = []
target_names = ['negative','positive']

while True:
    line = review_data.readline()
    if not line: break
    if line.split('\n')[0].split('\t')[2] == '0':
        target.append(0)
    else:
        target.append(1)
    data.append(line.split('\t')[1])
    
print(len(data),len(target))

33445 33445


In [64]:
import csv

#load polarity dictionary
file = open('./dic/polarity.csv','r',encoding='UTF-8')
reader = csv.reader(file)

#get header
header = reader.__next__()

#make dictionary
neg_dic = [[],[],[],[],[],[],[],[],[]]  #부정
pos_dic = [[],[],[],[],[],[],[],[],[]]  #긍정

for line in reader:
    line[0] = line[0].split(';') #split ngram
    maxValue = line[7]
    
    if maxValue == 'POS':
        for i in range(len(header)):
            pos_dic[i].append(line[i])
    elif maxValue == 'NEG':
        for i in range(len(header)):
            neg_dic[i].append(line[i])
            


def findPOS(unigram,bigram,trigram):
    pos_val = 0
    count=0
    for uni in unigram:
        if uni in pos_dic[0]:
            mvPos=pos_dic[0].index(uni) #max value position
            mvProp=pos_dic[8][mvPos] #max value prop
            pos_val += float(mvProp)
            count+=1
    for bi in bigram:
        if bi in pos_dic[0]:
            mvPos=pos_dic[0].index(bi) #max value position
            mvProp=pos_dic[8][mvPos] #max value prop
            pos_val += float(mvProp)
            count+=1
    for tri in trigram:
        if tri in pos_dic[0]:
            mvPos=pos_dic[0].index(tri) #max value position
            mvProp=pos_dic[8][mvPos] #max value prop
            pos_val += float(mvProp)
            count+=1
            
    return [pos_val,count]

def findNEG(unigram,bigram,trigram):
    neg_val = 0
    count=0
    for uni in unigram:
        if uni in neg_dic[0]:
            mvPos=neg_dic[0].index(uni) #max value position
            mvProp=neg_dic[8][mvPos] #max value prop
            neg_val += float(mvProp)
            count+=1
    for bi in bigram:
        if bi in neg_dic[0]:
            mvPos=neg_dic[0].index(bi) #max value position
            mvProp=neg_dic[8][mvPos] #max value prop
            neg_val += float(mvProp)
            count+=1
    for tri in trigram:
        if tri in neg_dic[0]:
            mvPos=neg_dic[0].index(tri) #max value position
            mvProp=neg_dic[8][mvPos] #max value prop
            neg_val += float(mvProp)
            count+=1
            
    return [neg_val,count]

In [178]:
def vectorize(sentence):
    wordsSet = {} #모든 단어의 카운팅 object
    
    import re
    import operator
    import numpy as np
    
    #모든 문자열 갯수 카운팅
    for term in sentence:
        for word in term.split(' '):
            if word in wordsSet :
                wordsSet[word]+=1
            else :
                wordsSet[word]=1
    
    #get all_words, 모든 단어의 배열
    #많이 사용된 상위 25개 제거
    all_words = []
    stop_words = []
    
    sortedWords = sorted(wordsSet.items(), key=operator.itemgetter(1))
    for i in range(len(sortedWords), len(sortedWords)):
        stop_words.append(sortedWords[i][0])     
    for i in range(0, len(sortedWords)):
        all_words.append(sortedWords[i][0])        
    all_words.sort()
    
    #wordsSet index
    wordsSet_index = {}
    i=0
    for word in all_words:
        wordsSet_index[word]=i
        i+=1

    #하나씩 돌면서 corpus_feature생성
    corpus_feature = np.zeros((len(sentence),len(all_words)+2))
    j=0
    for term in sentence:
        unigram = term.split(' ')
        bigram = []
        trigram = []
        for i in range(len(unigram)):
            if i==len(unigram)-1:
                break
            p = []
            p.append(unigram[i])
            p.append(unigram[i+1])
            bigram.append(p)
        for i in range(2,len(unigram)):
            p = []
            p.append(unigram[i-2])
            p.append(unigram[i-1])
            p.append(unigram[i])
            trigram.append(p)
        
        for word in unigram:
            if(word not in stop_words and word != ""):
                corpus_feature[j][wordsSet_index[word]]+=1
        
        #if negative append 0
        #elif positive append 1
        #print(findPOS(unigram,bigram,trigram))
        if(findPOS(unigram,bigram,trigram)[1] < findNEG(unigram,bigram,trigram)[1]):
            corpus_feature[j][len(corpus_feature[0])-1]=0
            corpus_feature[j][len(corpus_feature[0])-2]=1
        elif(findPOS(unigram,bigram,trigram)[1] > findNEG(unigram,bigram,trigram)[1]):
            corpus_feature[j][len(corpus_feature[0])-1]=1
            corpus_feature[j][len(corpus_feature[0])-2]=0
        j+=1
    
    return corpus_feature


In [179]:
corpus_feature = vectorize(data[:2000])
print(len(corpus_feature))
#split test case
from sklearn.model_selection import train_test_split
import numpy as np

#X_train, X_test, y_train, y_test = train_test_split(corpus_feature, target[:1000], test_size=0.2)

X_train=corpus_feature[:1500]
X_test=corpus_feature[1500:2000]
y_train=target[:1500]
y_test=target[1500:2000]

print(np.shape(corpus_feature))
print (len(X_train), len(X_test))



2000
(2000, 5484)
1500 500


In [180]:
# Learning
from sklearn.svm import SVC
clf = SVC(kernel = 'linear')
clf.fit(X_train, y_train)
#X_train과 y_train으로 SVC-linear로 학습

# Evaluation
y_pred = clf.predict(X_test)
#test셋으로 학습된 clf를 이용해서 예측데이터를 생성

from sklearn import metrics
print(metrics.classification_report(y_test, y_pred, target_names=target_names))

             precision    recall  f1-score   support

   negative       0.73      0.74      0.73       244
   positive       0.75      0.73      0.74       256

avg / total       0.74      0.74      0.74       500

